In [71]:
from pyspark import  SparkContext,SparkConf
sc = SparkContext( )

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-ffd50910b931>:2 

In [72]:
rawData = sc.textFile('/data/kaggle/train_noheader.tsv')
records = rawData.map(lambda x: x.split('\t'))
print records.first()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/hdp/2.5.0.0-1245/spark2/python/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/hdp/2.5.0.0-1245/spark2/python/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o20.sc

In [33]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

#过滤“符号，替换为空
trimmed = records.map(lambda item :[xx.replace('\"', '' ) for xx in item])
#获取最后一位
label = trimmed.map(lambda item: item[-1])
label = trimmed.map(lambda x : x[-1])
#数据处理，最后一位为key，从第四列开始-最后为value
#如果数值为？则置为0，如果为“ 置为空
#将x类型转换为整形，y的每一个值转为float类型
#将x和y构造成LabeledPoint结构
data = trimmed.map(lambda x : (x[-1],x[4:-1])).\
               map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in y])).\
               map(lambda (x,y):(x.replace("\"",""),[0.0 if yy =='?' else yy.replace("\"","") for yy in y])).\
               map(lambda (x,y):(int(x), [float(yy) for yy in y])).\
               map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))
print data.take(4)
print label.take(4)

[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]), LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]), LabeledPoint(1.0, [0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]), LabeledPoint(1.0, [0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0,120.0,5.0,0.041666667,0.100858369])]
[u'0', u'1', u'1', u'1']


In [9]:
data.cache()

PythonRDD[5] at RDD at PythonRDD.scala:48

In [10]:
data.count()

7395

In [49]:
ndata = trimmed.map(lambda x : (x[-1],x[4:-1])).\
               map(lambda (x,y):(x.replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in y])).\
               map(lambda (x,y):(x.replace("\"",""),[0.0 if yy =='?' else yy.replace("\"","") for yy in y])).\
               map(lambda (x,y):(int(x), [float(yy)  for yy in y])).\
               map(lambda (x,y):(int(x), [0 if yy<0 else yy for yy in y])).\
            map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))
ndata.take(4)


[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0,120.0,5.0,0.041666667,0.100858369])]

In [50]:
#训练模型
#导入相应的类

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.tree import DecisionTree

numIteration = 100
maxTreeDepth = 10

numClass = label.distinct().count() 
print "The class nums:", numClass
#训练逻辑回归、支持向量机、朴素贝叶斯和决策树模型
lrModel = LogisticRegressionWithSGD.train(data, numIteration)
svmModel = SVMWithSGD.train(data, numIteration)
nbModel = NaiveBayes.train(ndata)
print data.take(1)
dtModel = DecisionTree.trainClassifier(data, numClass, {}, impurity='entropy',maxDepth=maxTreeDepth)
print "LogisticRegressionWithSGD=", lrModel
print "SVMWithSGD=", svmModel
print "NativeBayes=", nbModel
print "DecisionTree=", dtModel
#逻辑回归的模型和svm的模型权重好像啊

The class nums: 2
[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])]
LogisticRegressionWithSGD= (weights=[-0.550608763357,-2.28562752154,-0.439331166963,-0.162228223433,-0.0292143091078,-0.0126114724381,-5.17235106843,0.207942323454,0.0,-0.129097852708,-0.0311426509567,-0.301522439418,-0.619933705283,-0.726666707707,-0.704681632452,-62.6756345609,-0.076132956751,-179.237173723,-99.5223009574,-6.87205576007,-0.191685436051,-0.159520259813], intercept=0.0)
SVMWithSGD= (weights=[-0.574551503125,-2.39015988071,-0.457233396112,-0.168485986067,-0.0298275974716,-0.0127395129754,-5.40743227025,0.217112420397,0.0,-0.13512838633,-0.0326565313643,-0.314805369155,-0.651196600765,-0.757096629994,-0.734668872816,-65.6071533534,-0.0794581311037,-163.709851544,-103.599245578,-7.17990782672,-0.20025471291,-0.166705251117], intercept=0.0)
NativeBay

In [53]:
#预测
testData = ndata.first()
print testData

(0.0,[0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575])


In [56]:
print lrModel.predict(testData.features), testData.label

0 0.0


In [62]:
predicts = svmModel.predict(data.map(lambda item:item.features))
print predicts.take(5), data.map(lambda label:label.label).take(5)

 [0, 0, 0, 0, 0] [0.0, 1.0, 1.0, 1.0, 0.0]


In [70]:
allCount = data.count()
lrCorrectCount = data.map(lambda item :1 if lrModel.predict(item.features)==item.label else 0).sum()
svmCorrectCount = data.map(lambda item: 1 if svmModel.predict(item.features) == item.label else 0).sum()
nbCorrectCount = ndata.map(lambda item: 1 if nbModel.predict(item.features)==item.label else 0).sum()
#决策树模型的预测正确数目
predictLabel= dtModel.predict(data.map(lambda point: point.features)).collect()
trueLabel = data.map(lambda point: point.label).collect()
dtCorrectCount = sum([1.0 if prediction == trueLabel[i] else 0.0 for i, prediction in enumerate(predictLabel)])
print "all=",allCount
print "lr=",lrCorrectCount/(allCount*1.0)
print "svm=", svmCorrectCount*1.0/allCount
print "nb=", nbCorrectCount*1.0/allCount
print "dt=", dtCorrectCount*1.0/allCount

all= 7395
lr= 0.486680189317
svm= 0.486680189317
nb= 0.580392156863
dt= 0.764435429344
